In [ ]:
import pandas as pd
pd.options.plotting.backend = 'plotly'

In [ ]:
vacc_data = pd.read_excel('../data/covid-19-vaccination-vaccination-data-22-december-2022.xlsx', index_col=0)['Value']

In [ ]:
agegroups = ['16-19'] + [f'{age}-{age + 4}' for age in range(20, 95, 5)] + ['95+']
doses = ['1+', '2+']
vacc_dict = {}
for dose in doses:
    s = 's' if dose == '2+' else ''
    vacc_dict[dose] = vacc_data[[f'Age group - {age} - Number of people who have received at least {dose[0]} dose{s}' for age in agegroups]].values
final_vacc_data = pd.DataFrame(
    vacc_dict, 
    index=agegroups
)
final_vacc_data['pop'] = vacc_data[[f'Age group - {age} - Population' for age in agegroups]].values
#     {
#         '1+': vacc_data[[f'Age group - {age} - Number of people who have received at least 1 dose' for age in agegroups]].values,
#         '2+': vacc_data[[f'Age group - {age} - Number of people who have received at least 2 doses' for age in agegroups]].values,
#         ,
#     },
#     index=agegroups,
# )

In [ ]:
age_strata = list(range(0, 80, 5))
modelled_vacc = pd.DataFrame(0.0, index=age_strata, columns=doses)
for dose in doses:
    modelled_vacc[dose][15] = final_vacc_data[dose]['16-19'] / final_vacc_data['pop']['16-19']
    for age in age_strata[4: -1]:
        modelled_vacc[dose][age] = final_vacc_data[dose][f'{age}-{age + 4}'] / final_vacc_data['pop'][f'{age}-{age + 4}']
    modelled_vacc[dose][75] = final_vacc_data.loc['75-79':, '1+'].sum() / final_vacc_data.loc['75-79':, 'pop'].sum()
    modelled_vacc[dose][modelled_vacc[dose] > 1.0] = 1.0
    s = 's' if dose == '2+' else ''
    modelled_vacc[dose][5] = vacc_data[f'National - Number of people 5-11 who have received at least {dose[0]} dose{s}'] / vacc_data['National - Population 5-11']
    modelled_vacc[dose][10] = vacc_data[f'National - Number of people 12-15 who have received at least {dose[0]} dose{s}'] / vacc_data['National - Population 12-15']

In [ ]:
modelled_vacc.plot()